## This assignment is designed for automated pathology detection for Medical Images in a relalistic setup, i.e. each image may have multiple pathologies/disorders. 
### The goal, for you as an MLE, is to design models and methods to predictively detect pathological images and explain the pathology sites in the image data.

## Data for this assignment is taken from a Kaggle contest: https://www.kaggle.com/c/vietai-advance-course-retinal-disease-detection/overview
Explanation of the data set:
The training data set contains 3435 retinal images that represent multiple pathological disorders. The patholgy classes and corresponding labels are: included in 'train.csv' file and each image can have more than one class category (multiple pathologies).
The labels for each image are

```
-opacity (0), 
-diabetic retinopathy (1), 
-glaucoma (2),
-macular edema (3),
-macular degeneration (4),
-retinal vascular occlusion (5)
-normal (6)
```
The test data set contains 350 unlabelled images.

# Label Propogation

### Setting up Label Propogation

## Task 3: Using the unlabelled data set in the 'test' folder augment the training data (semi-supervised learning) and report the variation in classification performance on test data set.(15 points)
[You may use any method of your choice, one possible way is mentioned below.] 

```
Hint: 
a. Train a model using the 'train' split.
b. Pass the unlabelled images through the trained model and retrieve the dense layer feature prior to classification layer. Using this dense layer as representative of the image, apply label propagation to retrieve labels correspndng to the unbalelled data.
c. Next, concatenate the train data with the unlabelled data (that has now been self labelled) and retrain the network.
d. Report classification performance on test data
Use the unlabelled test data  to improve classification performance by using a semi-supervised label-propagation/self-labelling approach. (20 points)
```

# Task 3: Label Prop

In [2]:
# importing the required modules
import pandas as pd
import numpy as np
import cv2
from PIL import Image, ImageEnhance
import os
import shutil

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#from google.colab.patches import cv2_imshow

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.backend import gradients, mean

from sklearn.model_selection import train_test_split

In [3]:
# find path for the data
data_path = 'Data'

In [16]:
# read the training labels
train_labels = pd.read_csv(os.path.join(data_path, 'train/train.csv'))
train_labels.set_index('filename', inplace=True)
train_labels.head()

,opacity,diabetic retinopathy,glaucoma,macular edema,macular degeneration,retinal vascular occlusion,normal
filename,,,,,,,
c24a1b14d253.jpg,0,0,0,0,0,1,0
9ee905a41651.jpg,0,0,0,0,0,1,0
3f58d128caf6.jpg,0,0,1,0,0,0,0
4ce6599e7b20.jpg,1,0,0,0,1,0,0
0def470360e4.jpg,1,0,0,0,1,0,0


### Create Data Pipelines

In [12]:
# function to load images to numpy array and normalize
def img_to_np(img_path):
    # open image using PIL
    img_temp = Image.open(img_path)
    img_temp = img_temp.resize((IMAGE_SIZE,IMAGE_SIZE), resample=1)

    # convert to np array
    img_temp = np.array(img_temp).astype(np.float32)
    img_temp = img_temp/255.0
    
    return img_temp

In [14]:
# define image size and channels
IMAGE_SIZE = 224
CHANNELS = 3

BATCH_SIZE = 64

In [4]:
# direcotries containing files
test_dir = 'Data/test'

In [5]:
# load trained model
model_pretrained = tf.keras.models.load_model('models/dbr_v1.h5')
model_pretrained.trainable = False

In [6]:
model_pretrained.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 222, 222, 256 7168        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 111, 111, 256 0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 109, 109, 256 590080      max_pooling2d_8[0][0]            
_______________________________________________________________________________________

In [7]:
# get last dense layer
last_dense_layer = 'dense_13'

In [8]:
# setup to get the output from the last dense layer
last_layer = model_pretrained.get_layer(last_dense_layer)
last_conv_layer_model = tf.keras.Model(model_pretrained.inputs, last_layer.output)

In [9]:
# creating a train array for passing to knn
pred_train = np.array([ 'name', 'p1', 'p2', 'labels', 'label_num'])
pred_train = np.expand_dims(pred_train, axis=0)
pred_train.shape

(1, 5)

In [10]:
train_original_dir = 'Data/train/train'

In [17]:
# iterating over the image folder to collect predictions, names and labels
for pic in os.listdir(train_original_dir):
    # get image array
    im_temp = img_to_np(os.path.join(train_original_dir, pic))
    im_temp = np.expand_dims(im_temp, axis=0)

    pred_temp = []

    # get image label
    max_val = 1
    labels = [i for i in train_labels.loc[pic].values]

    if labels[1] == 1:
        pred_temp.append(pic)
        for j in last_conv_layer_model.predict(im_temp)[0]:
            pred_temp.append(j)
        pred_temp.append('diabetic retinopathy')
        pred_temp.append(1)
    elif labels[2] == 1:
        continue
    #    pred_temp.append(pic)
    #    for j in last_conv_layer_model.predict(im_temp)[0]:
    #        pred_temp.append(j)
    #    pred_temp.append('glaucoma')
    #    pred_temp.append(2)
    elif labels[6] == 1:
        pred_temp.append(pic)
        for j in last_conv_layer_model.predict(im_temp)[0]:
            pred_temp.append(j)
        pred_temp.append('normal')
        pred_temp.append(2)
    else:
        continue
        #pred_temp.append(pic)
        #for j in last_conv_layer_model.predict(im_temp)[0]:
        #    pred_temp.append(j)
        #pred_temp.append('other')
        #pred_temp.append(4)

    pred_train = np.append(pred_train, np.expand_dims(np.array(pred_temp), axis=0), axis=0)

In [18]:
# extract specific columns from np array
# use X = data[:, [1, 9]]
X_train = pred_train[1:,[1, 2]]

In [19]:
# extract targets
y_train = pred_train[1:, [4]]

In [20]:
from sklearn.neighbors import KNeighborsClassifier as KNN

In [26]:
# create a label propagation model
prop = KNN(n_neighbors=2)

In [27]:
# fit the model
prop.fit(X_train.astype(np.float32), y_train.ravel())

KNeighborsClassifier(n_neighbors=2)

In [28]:
# predict labels for the test folder
# creating a test array for passing to knn model
pred_test = np.array(['name', 'labels', 'label_num'])
pred_test = np.expand_dims(pred_test, axis=0)
pred_test.shape

(1, 3)

In [29]:
# create labels for the test images
# iterating over the image folder to collect predictions, names and labels
for pic in os.listdir(test_dir):
    # get image array
    im_temp = img_to_np(os.path.join(test_dir, pic))
    im_temp = np.expand_dims(im_temp, axis=0)

    pred_temp = []

    # get image predited labels
    label_predicted = prop.predict(np.expand_dims(
        last_conv_layer_model.predict(im_temp)[0], axis=0))
  
    # append labels to the new array
    if int(label_predicted[0]) == 1:
        pred_temp.append(pic)
        pred_temp.append('diabetic retinopathy')
        pred_temp.append(1)

    #elif int(label_predicted[0]) == 2:
    #    pred_temp.append(pic)
    #    pred_temp.append('galucoma')
    #    pred_temp.append(2)

    elif int(label_predicted[0]) == 2:
        pred_temp.append(pic)
        pred_temp.append('normal')
        pred_temp.append(3)

    else:
        continue
        #pred_temp.append(pic)
        #pred_temp.append('other')
        #pred_temp.append(4)

    pred_test = np.append(pred_test, np.expand_dims(np.array(pred_temp), axis=0), axis=0)

In [ ]:
pred_test[:, [2]]

### Append the test data to train data

In [30]:
pred_test_ = pred_test[1:]
pred_test_.shape

(350, 3)

In [44]:
# create new folders with images
new_image_dir = 'Data/separated_pics_aug'

for row in pred_test_:
    file_name = row[0]
    label_num = int(row[2])

    if label_num == 1:
        shutil.copy(os.path.join(test_dir, file_name), 
                    os.path.join(new_image_dir, 'diabetic retinopathy/'+file_name))
  
    #elif label_num == 2:
    #    shutil.copy(os.path.join(test_img_dir, file_name), 
    #                os.path.join(new_image_dir, 'glaucoma/'+file_name))
    
    elif label_num == 3:
        shutil.copy(os.path.join(test_dir, file_name), 
                    os.path.join(new_image_dir, 'normal/'+file_name))
  
    #else:
    #    shutil.copy(os.path.join(test_img_dir, file_name), 
    #                os.path.join(new_image_dir, 'other/'+file_name))

## Train on Concatenated Data

In [45]:
# check if the data has been added
print(len(os.listdir('Data/separated_pics_train/diabetic retinopathy')))
print(len(os.listdir('Data/separated_pics_aug/diabetic retinopathy')))
print(len(os.listdir('Data/separated_pics_train/glaucoma')))
print(len(os.listdir('Data/separated_pics_aug/glaucoma')))
print(len(os.listdir('Data/separated_pics_train/normal')))
print(len(os.listdir('Data/separated_pics_aug/normal')))
print(len(os.listdir('Data/separated_pics_train/other')))
print(len(os.listdir('Data/separated_pics_aug/other')))

755
1057
578
578
525
573
749
1099


In [343]:
# define image size and channels
IMAGE_SIZE = 256
CHANNELS = 3

BATCH_SIZE = 64

In [344]:
# training image dir
train_img_dir = os.path.join(data_path, 'separated_pics_aug')

# testing image dir
test_img_dir = 'Data/test'

In [349]:
train_datagen = ImageDataGenerator(
    rotation_range=10.,
    width_shift_range=0.2,
    height_shift_range=0.2,
    channel_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(-0.5,0.5),
    fill_mode='constant',
    data_format="channels_last",
    #preprocessing_function=tf.keras.applications.vgg19.preprocess_input,
    preprocessing_function=contrast,
    validation_split = 0.3,
    rescale=1.0/255.0
)

In [350]:
# create training data
train_gen = train_datagen.flow_from_directory(
    directory=train_img_dir,
    shuffle=True, 
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

Found 2072 images belonging to 4 classes.


In [351]:
# create training data
val_gen = train_datagen.flow_from_directory(
    directory=train_img_dir,
    shuffle=True, 
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 885 images belonging to 4 classes.


### Creating callback for saving models and stopping training if certain accuracy is reached.

Download application from the tf.keras.application

In [317]:
# importing resnet50 from tensorflow applications
base_model = tf.keras.applications.ResNet50(
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False,
    weights='imagenet')

In [318]:
base_model.trainable = True
#base_model.summary()

In [319]:
# select a layer to start optimizing from, 
# here we will train only the later half of the model
train_layer = base_model.get_layer('conv4_block4_1_conv')

# get train_layer index
layer_index = base_model.layers.index(train_layer)

In [320]:
# make layers before the training layer non trainable
for layer in base_model.layers[:layer_index]:
    layer.trainable = False

Skipping the final activation layer and just getting the logits as these will be used for the final part of the assignment

In [321]:
# define model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_16  (None, 2048)              0         
_________________________________________________________________
dense_46 (Dense)             (None, 4)                 8196      
Total params: 23,595,908
Trainable params: 18,340,356
Non-trainable params: 5,255,552
_________________________________________________________________


In [322]:
# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics='accuracy')

In [ ]:
# train model
history = model.fit(train_gen,
                    steps_per_epoch=train_gen.samples // BATCH_SIZE,
                    epochs= 100,
                    verbose=1,
                    validation_data=val_gen,
                    validation_steps=val_gen.samples // BATCH_SIZE,
                    callbacks=[accuracy_callback])
                    #callbacks=[stop_train, accuracy_callback, lr_decay])

Epoch 1/100
32/32 [==============================] - 107s 3s/step - loss: 1.3753 - accuracy: 0.3327 - val_loss: 435.1318 - val_accuracy: 0.3678
Epoch 2/100
32/32 [==============================] - 90s 3s/step - loss: 1.3614 - accuracy: 0.3675 - val_loss: 29.0607 - val_accuracy: 0.3798
Epoch 3/100
32/32 [==============================] - 68s 2s/step - loss: 1.3663 - accuracy: 0.3635 - val_loss: 13.4626 - val_accuracy: 0.3738
Epoch 4/100
32/32 [==============================] - 90s 3s/step - loss: 1.3590 - accuracy: 0.3635 - val_loss: 2.0717 - val_accuracy: 0.2464
Epoch 5/100
32/32 [==============================] - 70s 2s/step - loss: 1.4009 - accuracy: 0.3277 - val_loss: 1.3582 - val_accuracy: 0.3702
Epoch 6/100
32/32 [==============================] - 69s 2s/step - loss: 1.3684 - accuracy: 0.3242 - val_loss: 1.8668 - val_accuracy: 0.3750
Epoch 7/100
32/32 [==============================] - 70s 2s/step - loss: 1.3624 - accuracy: 0.3561 - val_loss: 1.4865 - val_accuracy: 0.3810
Epoch 8/

In [ ]:
# plottig history
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([1, 0])
plt.legend(loc='upper right')
plt.show()

In [ ]:
model.save('models/resnet50_augumented.h5')